In [31]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchsummary

In [2]:
dtype = torch.FloatTensor

In [3]:
# Text-CNN Parameters
embedding_size = 2 # n-gram
sequence_length = 3 
num_classes = 2
filter_sizes = [2, 2, 2] # n-gram window
num_filters = 3

In [4]:
# words sentences (sequence_length=3)
sentences = [
    "i love you",
    "he loves me",
    "she likes baseball",
    "i hate you",
    "sorry for that",
    "this is awful"
]

labels = [1, 1, 1, 0, 0, 0]

In [7]:
%%time
word_list = " ".join(sentences).split()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.91 µs


In [10]:
word_list = list(set(word_list))

In [12]:
word2idx = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word2idx); vocab_size

16

In [20]:
inputs = []
for s in sentences:
    inputs.append(np.asarray([word2idx[word] for word in s.split()]))

In [22]:
targets = []
for out in labels:
    targets.append(out) # to using Torch Softmax Loss Function

In [24]:
input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))

In [28]:
input_batch

tensor([[12,  5,  2],
        [ 6,  1,  9],
        [ 3, 15, 10],
        [12, 11,  2],
        [ 8, 14, 13],
        [ 0,  4,  7]])

In [26]:
target_batch

tensor([1, 1, 1, 0, 0, 0])

In [54]:
# Text CNN module
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        
        self.num_filters_total = num_filters * len(filter_sizes)
        self.embedding = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype) #embedding matrix
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype)
        self.bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)
        
    def forward(self, X):
        embedded_chars = self.embedding[X] # [batch_size, ]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel, [batch, channel(=1), sequence_length, embedding_size]
        
        pooled_outputs = []
        for filter_size in filter_sizes: # [2,2,2]
            # conv: [input_channel(1), output_channel(3), (height, width), bias option]
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)
            h = F.relu(conv)
            # max_pool : ((filter_height, filter_width))
            max_pool = nn.MaxPool2d((sequence_length - filter_size + 1, 1))
            # pooled = [batch_size(6), output_height(1), output_width(1), output_channel(3)*3]
            pooled = max_pool(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)
        
        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3) * 3]
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel * 3)]

        model = torch.mm(h_pool_flat, self.Weight) + self.bias # [batch_size, num_classes]
        return model

In [62]:
model = TextCNN()

In [63]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [64]:
# Training
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)
    
    # output: [batch_size, num_classes], target_batch: [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch+1) % 500 == 0:
        print('Epoch: ', '%04d' % (epoch+1), 'loss: ' '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch:  0500 loss: 0.601695
Epoch:  1000 loss: 0.499576
Epoch:  1500 loss: 0.291872
Epoch:  2000 loss: 0.172124
Epoch:  2500 loss: 0.090380
Epoch:  3000 loss: 0.117862
Epoch:  3500 loss: 0.110925
Epoch:  4000 loss: 0.054028
Epoch:  4500 loss: 0.071319
Epoch:  5000 loss: 0.013442


In [66]:
# Test
test_text = "sorry you awful"
tests = [np.asarray([word2idx[word] for word in test_text.split()])]; tests

[array([8, 2, 7])]

In [68]:
test_batch = Variable(torch.LongTensor(tests)); test_batch.shape

torch.Size([1, 3])

In [81]:
# Predict
pred = model(test_batch)
pred = pred.data.max(1, keepdim=True)[1]

In [90]:
mean = ""
if pred[0][0] == 0:
    mean = " is bad mean"
else:
    mean = " is good mean"
print(f'[{test_text}]' + mean)

[sorry you awful] is bad mean
